In [ ]:
import sys
import os
sys.path.append(os.path.abspath("../src"))
from Confocal_GUI.live_plot import change_to_inline, change_to_nbagg, enable_long_output, DataFigure
from Confocal_GUI.logic import pl, live, odmr, GUI, rabi, spinecho, ramsey, t1, roduration
from Confocal_GUI.device import USB6346, DSG836, initialize_classes, Pulse
import numpy as np
import matplotlib
import matplotlib.pyplot as plt

%load_ext autoreload
%autoreload 2

In [ ]:
import pyvisa
rm = pyvisa.ResourceManager()
rm.list_resources()
# get visa address of DSG836 here
# and replace 'USB0::0x1AB1::0x099C::DSG8M223900103::INSTR' with the correct address you get from here

In [ ]:
# initialize all classes and functions, bind them to devices wanted
enable_long_output()
# make sure that GUI() can output fig to jupyter notebook cell without being scrolled
port_config = {'analog_signal':'ai0', 'analog_gate':'ai1', 'analog_gate_ref':'ai2',\
                           'apd_signal':'PFI3', 'apd_gate':'PFI4', 'apd_gate_ref':'PFI5'}

config = {    
    'counter': {'type': 'USB6346', 'port_config':port_config},    
    'scanner': {'type': 'USB6346'},    
    'rf': {'type': 'DSG836', 'visa_str'='USB0::0x1AB1::0x099C::DSG8M223900103::INSTR'},
    'pulse': {'type': 'Pulse'},
}
# scanner, counter, etc. 
# are default keys for all functions to call objetive, 'type' defines bind which class
# or function to the key.

config_instances = initialize_classes(config, lookup_dict=globals(), namespace=globals())
config_instances = {**config_instances, **{'display_scale':1}}

In [ ]:
# refer to 'examples with virtual devices.ipynb' for a complete guide on basic use of this project

<h1 style="color: blue;">cw-ODMR</h1>

In [ ]:
help(pulse.gui)

In [ ]:
# check you pulse sequence and check your BNC/SMA cables connections
# make sure you hit the On Pulse button otherwise no gate signal for DAQ
pulse.load_from_file('../src/Confocal_GUI/device/cwODMR_pulse*')
pulse.gui()

In [ ]:
help(odmr)

In [ ]:
fig, data_figure = odmr(data_x=np.arange(2.88-0.1, 2.88+0.1, 0.001), exposure=0.1,\
                            power=-10,\
                            config_instances=config_instances, repeat=1, is_GUI=False,\
                            counter_mode='analog', data_mode='single', relim_mode='tight')

In [ ]:
# or use the GUI version
fig, data_figure = odmr(data_x=np.arange(2.88-0.1, 2.88+0.1, 0.001), exposure=0.1,\
                            power=-10,\
                            config_instances=config_instances, repeat=1, is_GUI=True,\
                            counter_mode='analog', data_mode='single', relim_mode='tight')

In [ ]:
# zoom in into one dip and find the peak position using data_figure.lorent()
# or just hit 'fit' button in GUI
_, popt = data_figure.lorent()

In [ ]:
# change RF power using 
rf.power = -20

In [ ]:
# or using the gui
rf.gui()

In [ ]:
# load another pulse sequence with reference data
# make sure you hit the On Pulse button otherwise no gate signal for DAQ
pulse.load_from_file('../src/Confocal_GUI/device/cwODMR_ref_pulse*')
pulse.gui()

In [ ]:
# or a easier way to design pulse sequence
pulse.load_from_file('../src/Confocal_GUI/device/cwODMR_ref_auto_pulse*')
pulse.gui()

In [ ]:
# see how is reference data different from main data
fig, data_figure = odmr(data_x=np.arange(2.88-0.1, 2.88+0.1, 0.001), exposure=0.1,\
                            power=-10,\
                            config_instances=config_instances, repeat=1, is_GUI=False,\
                            counter_mode='analog', data_mode='dual', relim_mode='tight')

In [ ]:
# see how subtraction/division can help reducing the drift of signal
fig, data_figure = odmr(data_x=np.arange(2.88-0.1, 2.88+0.1, 0.001), exposure=0.1,\
                            power=-10,\
                            config_instances=config_instances, repeat=1, is_GUI=False,\
                            counter_mode='analog', data_mode='ref_sub', relim_mode='tight')

<h1 style="color: blue;">Rabi</h1>

In [ ]:
help(rabi)

In [ ]:
# try default
# if it does not work well, try editting pulse sequence
fig, data_figure = rabi(data_x=np.arange(20, 2000, 10), exposure=0.1, power=-10, frequency=2.88, 
                        pulse_file=None,
                        config_instances=config_instances, repeat=1, is_GUI=True,
                        counter_mode='analog', data_mode='ref_sub', relim_mode='tight')

In [ ]:
# default pulse sequence for rabi
pulse.load_from_file('../src/Confocal_GUI/device/Rabi_pulse*')
pulse.gui()

In [ ]:
# fit and get Rabi frequency
# or just hit 'fit' button in GUI
_, popt = data_figure.rabi()

<h1 style="color: blue;">Others</h1>

In [ ]:
help(ramsey)

In [ ]:
help(spinecho)

In [ ]:
help(t1)

In [ ]:
help(roduration)